# Agent Chat UI: the new Toy Front-End for LangChain 1.0 Agents

## Introduction

If you're learning LangChain 1.0 and have been building agents that run in your terminal or Jupyter notebooks, you might be wondering: "How can I actually see my agent working in a real chat interface?" The good news is that LangChain has created a free, ready-to-use tool called **Agent Chat UI** that gives your agents a professional-looking chat interface with almost no additional coding required.

Think of Agent Chat UI as a "toy front-end" – a simple but functional web interface that lets you interact with your LangChain agents through a familiar chat window, just like ChatGPT or Claude. This is perfect for:

- **Testing your agents** in a more realistic environment
- **Demonstrating your work** to classmates, instructors, or potential employers
- **Understanding how agents work** by seeing their responses in real-time
- **Learning web integration** without having to build your own frontend from scratch

## What is Agent Chat UI?

Agent Chat UI is a Next.js web application created by the LangChain team. It provides a chat interface that can connect to any LangGraph server (LangGraph is the framework you use to build agents in LangChain 1.0).

**Important clarification:** In LangChain 1.0, *agents are built with LangGraph*. LangChain provides models, tools, and abstractions—but LangGraph is what turns them into **stateful, server-ready agents** that UIs like Agent Chat UI can talk to. Think of LangChain as your toolbox and LangGraph as the workshop where you assemble those tools into a working agent.

The interface displays:

- Your messages and the agent's responses
- Tool calls that your agent makes (like web searches, API calls, etc.)
- Streaming responses as the agent thinks and responds
- A clean, professional chat design

Best of all, it's completely free and open-source.

## What Agent Chat UI Is *Not*

Before we dive into setup, let's set clear expectations about what Agent Chat UI is designed for:

* It is **not** a full product UI ready for end users
* It is **not** optimized for production deployment without modification
* It is **not** meant to replace a custom frontend for a real application

Think of it as a **debugging, learning, and demo tool**—not a production chat app. It's perfect for development and testing, but if you're building something for actual users, you'll eventually want to create a custom interface. For learning LangChain 1.0, however, Agent Chat UI is exactly what you need.

## Prerequisites: What You Need

Before using Agent Chat UI, you should have:

1. **A working LangChain 1.0 agent** – You should have already built an agent using LangGraph that runs successfully in your Python environment
2. **Basic Python knowledge** – Understanding of how to run Python scripts and manage virtual environments
3. **Node.js installed** – Agent Chat UI is a JavaScript application, so you'll need Node.js (version 18 or higher)
4. **A LangSmith account** (optional for local testing; required if you deploy via LangGraph Cloud or want hosted observability) – Free tier is sufficient

## Two Ways to Use Agent Chat UI

There are two approaches to using Agent Chat UI, depending on your needs:

#### Option 1: Use the Deployed Version (Easiest)

The LangChain team hosts a live version at **agentchat.vercel.app**. This is the fastest way to get started if you want to connect to a deployed LangGraph server.

**Pros:**
- No installation required
- Works immediately in your browser
- Great for quick testing

**Cons:**
- Requires your agent to be deployed to LangGraph Cloud
- Not ideal for local development and learning

#### Option 2: Run Agent Chat UI Locally (Recommended for Learning)

This option lets you run the chat interface on your own computer and connect it to agents running locally. This is the best approach for students because:

- You can see both your agent code and the chat interface
- You can make changes and test immediately
- You don't need to deploy anything to the cloud
- It helps you understand how frontend and backend connect


## Step-by-Step Guide: Setting Up Agent Chat UI Locally

Let's walk through the complete process of getting Agent Chat UI working with your local LangChain agent.


#### Step 1: Prepare Your LangChain Agent

First, make sure your agent is set up as a LangGraph server. Your agent code should look something like this:

```python
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

# Define your agent
# Note: Use any supported model - gpt-4o-mini is cost-effective for learning
model = ChatOpenAI(model="gpt-4o-mini")
tools = [your_tools_here]  # Any tools your agent uses

# Create the agent - the new LangChain 1.0 way
agent = create_agent(
    model=model,
    tools=tools,
    system_prompt="You are a helpful assistant."
)

# This is important: your agent needs to be runnable as a server
```

**Key requirement:** Your agent must have a state that includes a `messages` key. This is how Agent Chat UI knows where to find the conversation history.

#### The One Contract You Must Respect

Agent Chat UI is not a generic frontend—it expects your agent to behave in a specific way. This is the most important thing to understand:

**The Messages Contract:**
* Your graph state **must** contain a `messages` list
* Each conversation turn **must append** messages to that list
* Responses must be returned as `{"messages": [...]}`

If this contract is broken, the UI cannot render anything—even if your agent logic is perfectly correct. Many beginners experience a blank screen because their agent returns data in a different format. 

When you use `create_agent` from LangChain 1.0, this contract is automatically handled for you. The function builds on LangGraph's runtime under the hood and ensures the proper state structure is created.

If you're building a custom graph from scratch (not using `create_agent`), make sure your state definition includes:

```python
from langgraph.graph import MessagesState

# Your graph state should inherit from MessagesState
# or include a messages field with the proper annotation
```


#### Step 2: Start Your LangGraph Server

LangGraph provides a built-in server that makes your agent accessible over HTTP. Here's how to start it:

1. Make sure you have LangGraph CLI installed:

```bash
pip install langgraph-cli
```

2. Create a `langgraph.json` file in your project directory:

```json
{
  "dependencies": ["."],
  "graphs": {
    "agent": "./your_agent_file.py:agent"
  }
}
```

This file tells LangGraph where your agent code is located.

3. Start the server:

```bash
langgraph dev
```

You should see output like:

```
🚀 API: http://127.0.0.1:2024
🎨 Studio UI: https://smith.langchain.com/studio/?baseUrl=http://127.0.0.1:2024
```

**Important:** Write down the API URL (typically `http://127.0.0.1:2024`) and the graph ID (in our example, it's `agent`). You'll need these in the next step.

#### Step 3: Install and Run Agent Chat UI

Now let's get the chat interface running:

1. **Quick installation using npx** (recommended):

```bash
npx create-agent-chat-app
```

This command will:
- Ask you a few questions about your setup
- Download and install everything you need
- Create a new folder with the Agent Chat UI

Alternatively, you can clone the repository manually:

```bash
git clone https://github.com/langchain-ai/agent-chat-ui.git
cd agent-chat-ui
pnpm install
```

2. **Start the chat interface:**

```bash
pnpm dev
```

3. **Open your browser** and go to `http://localhost:3000`

#### Step 4: Connect the Chat Interface to Your Agent

When you open `http://localhost:3000` for the first time, you'll see a setup form asking for:

1. **Deployment URL:** Enter `http://localhost:2024` (or whatever URL your LangGraph server showed)
2. **Assistant/Graph ID:** Enter `agent` (or the ID you specified in your `langgraph.json`)
3. **LangSmith API Key:** Leave this blank for local testing

Click "Continue" and you should now see a chat interface!

#### Step 5: Test Your Agent

Type a message in the chat box and press Enter. You should see:

- Your message appear in the chat
- The agent's response streaming in real-time
- Any tool calls the agent makes displayed as they happen (note: tool call panels only appear if your agent actually uses tools—a plain conversational agent won't show these sections)

If something goes wrong, check that:
- Your LangGraph server is still running
- The URLs and IDs match exactly
- Your agent code doesn't have any errors
- Your agent's state includes the `messages` key (see "The One Contract You Must Respect" section above)


## Making Development Easier: Using Environment Variables

Instead of entering the connection details every time, you can set up environment variables:

1. In the `agent-chat-ui` folder, create a file named `.env`:

```env
NEXT_PUBLIC_API_URL=http://localhost:2024
NEXT_PUBLIC_ASSISTANT_ID=agent
```

2. Restart the Agent Chat UI:

```bash
pnpm dev
```

Now it will automatically connect to your agent without asking for details!


## Understanding What's Happening Behind the Scenes

When you type a message in Agent Chat UI, here's what happens:

1. **Message sent:** Your message is sent from the web browser to the Next.js server
2. **Forwarded to agent:** The Next.js server forwards your message to your LangGraph server
3. **Agent processes:** Your LangChain agent receives the message and starts processing
4. **Streaming response:** As your agent thinks and generates responses, these are streamed back through the LangGraph server
5. **Real-time display:** Agent Chat UI receives the streaming data and displays it in the chat window

This architecture means your Python agent code doesn't need to know anything about web development – it just processes messages and returns responses, while Agent Chat UI handles all the web interface details.


## Customizing the Chat Interface

#### Hiding Internal Messages

Sometimes your agent might have internal reasoning steps you don't want to show in the chat. You can hide these by adding tags to your model:

```python
from langchain_openai import ChatOpenAI

# This model's outputs won't stream in the UI
model = ChatOpenAI(model="gpt-4o-mini").with_config(
    config={"tags": ["langsmith:nostream"]}
)

# Works with other providers too:
# from langchain_anthropic import ChatAnthropic
# model = ChatAnthropic(model="claude-3-5-sonnet-20241022").with_config(
#     config={"tags": ["langsmith:nostream"]}
# )
```

#### Controlling Message Display

To completely hide a message from the UI, prefix its ID with `do-not-render-`:

```python
result = model.invoke([messages])
result.id = f"do-not-render-{result.id}"
return {"messages": [result]}
```

This is useful for internal agent reasoning that shouldn't be shown to users.


## Common Issues and Solutions

#### Issue 1: "Connection Failed" Error

**Solution:** Make sure your LangGraph server is running. In your terminal where you ran `langgraph dev`, you should see logs of incoming requests. If not, restart the server.

#### Issue 2: Chat Interface Loads But Shows No Response

**Solution:** Check that your agent's state includes a `messages` key. Agent Chat UI specifically looks for this key to display the conversation.

#### Issue 3: Tool Calls Not Showing

**Solution:** Make sure your tools are properly configured in your agent. Agent Chat UI automatically displays tool calls, but only if they're properly formatted in the agent's output.

#### Issue 4: "Port Already in Use" Error

**Solution:** Either:
- Stop any other process using port 3000 (for the chat UI) or port 2024 (for the LangGraph server)
- Or specify a different port: `pnpm dev -- -p 3001`


## Best Practices for Using Agent Chat UI

1. **Start simple:** Begin with a basic agent that just responds to messages, then add complexity
2. **Test incrementally:** Make small changes to your agent and test them immediately in the chat
3. **Watch the logs:** Keep an eye on your terminal where LangGraph is running – error messages there are very helpful
4. **Use the browser console:** Press F12 in your browser to see additional debugging information
5. **Iterate quickly:** The beauty of local development is you can change your agent code, restart the server, and test again in seconds


## Next Steps: Going Beyond Local Testing

Once your agent works well locally with Agent Chat UI, you might want to:

#### Deploy Your Agent

LangChain offers LangGraph Cloud for deploying your agents. This makes them accessible from anywhere, not just your local machine.

## Conclusion

Agent Chat UI transforms your terminal-based LangChain agents into interactive chat applications with minimal effort. As a student learning LangChain 1.0, this tool gives you:

- A professional way to demonstrate your work
- Immediate visual feedback on how your agents behave
- A bridge between backend Python code and frontend web interfaces
- A foundation for building more sophisticated applications

Remember, the goal isn't just to have a pretty interface – it's to understand how agents work in a more intuitive way and to build your skills in creating AI applications that people can actually use.

The Agent Chat UI is your "toy front-end," but it's a powerful toy that can help you learn, test, and showcase your LangChain 1.0 agents effectively. Start simple, experiment often, and don't be afraid to break things – that's how you learn!


## Quick Troubleshooting Checklist

If your Agent Chat UI isn't working, check these five things in order:

1. ✅ **Is your LangGraph server running?** Look for the terminal window where you ran `langgraph dev`
2. ✅ **Does your agent have a `messages` key in its state?** This is the most common issue for beginners
3. ✅ **Are the URLs correct?** Deployment URL should match your LangGraph server (usually `http://localhost:2024`)
4. ✅ **Is the Assistant/Graph ID correct?** It should match what's in your `langgraph.json` file
5. ✅ **Any errors in the browser console?** Press F12 and check the Console tab for error messages

If you've checked all five and it still doesn't work, review the "Common Issues and Solutions" section above.

## Additional Resources

- **Official Agent Chat UI Repository:** https://github.com/langchain-ai/agent-chat-ui

# OK. Let's see the Agent Chat UI at work with a basic example.

## The Code

In [1]:
"""
Simple LangChain 1.0 Agent for Learning
This agent can do math, tell jokes, and have conversations.
Updated to use the new create_agent function from LangChain 1.0
"""

import os
from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

# Load environment variables from .env file
load_dotenv()

# Check if API key is loaded
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("Please set OPENAI_API_KEY in your .env file")

# =============================================================================
# STEP 1: Define Tools (Functions your agent can use)
# =============================================================================

@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers together.
    
    Args:
        a: First number
        b: Second number
    
    Returns:
        The product of a and b
    """
    return a * b

@tool
def add(a: float, b: float) -> float:
    """Add two numbers together.
    
    Args:
        a: First number
        b: Second number
    
    Returns:
        The sum of a and b
    """
    return a + b

@tool
def divide(a: float, b: float) -> float:
    """Divide two numbers.
    
    Args:
        a: Numerator (number to be divided)
        b: Denominator (number to divide by)
    
    Returns:
        The result of a divided by b
    """
    if b == 0:
        return "Error: Cannot divide by zero!"
    return a / b

@tool
def tell_joke() -> str:
    """Tell a programming joke.
    
    Returns:
        A funny programming joke
    """
    jokes = [
        "Why do programmers prefer dark mode? Because light attracts bugs!",
        "Why do Java developers wear glasses? Because they don't C#!",
        "How many programmers does it take to change a light bulb? None, that's a hardware problem!",
        "Why did the programmer quit his job? Because he didn't get arrays!",
    ]
    import random
    return random.choice(jokes)

# =============================================================================
# STEP 2: Create the Language Model
# =============================================================================

# Using GPT-4o-mini because it's cost-effective for learning
# You can also use: "gpt-4o", "gpt-3.5-turbo", etc.
model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0  # Controls randomness (0=focused, 1=creative)
)

# =============================================================================
# STEP 3: Combine Tools into a List
# =============================================================================

tools = [multiply, add, divide, tell_joke]

# =============================================================================
# STEP 4: Create the Agent (NEW LANGCHAIN 1.0 WAY)
# =============================================================================

# create_agent is the new standard in LangChain 1.0
# - Replaces the deprecated create_react_agent from langgraph.prebuilt
# - Uses system_prompt instead of state_modifier (clearer naming)
# - Still builds on LangGraph runtime under the hood
# - Handles the message flow automatically
# - Creates the proper state structure (with the 'messages' key)

agent = create_agent(
    model=model,
    tools=tools,
    system_prompt="You are a helpful assistant that can do math and tell jokes. "
                  "Always be friendly and clear in your responses."
)

# =============================================================================
# WHAT'S HAPPENING UNDER THE HOOD
# =============================================================================
# The 'agent' variable now contains a complete LangGraph compiled graph:
# 
# 1. It has a STATE that tracks the conversation (messages list)
# 2. It has NODES:
#    - "agent" node: Where the LLM thinks and decides what to do
#    - "tools" node: Where tools actually execute
# 3. It has EDGES connecting these nodes
# 4. It follows the ReAct pattern: Reason → Act → Observe (repeat)
#
# Agent Chat UI expects this specific structure with the 'messages' key!
# The create_agent function ensures this contract is met automatically.
# =============================================================================

# This allows the agent to be imported by other files
if __name__ == "__main__":
    print("Agent is ready! Use 'langgraph dev' to start the server.")

Agent is ready! Use 'langgraph dev' to start the server.


## Let's explain the previous code in simple terms and the steps we will need to take next

We have created a **Weather & Math Assistant** that can:
- Answer math questions
- Tell jokes
- Have normal conversations

This is perfect for learning because it's simple but demonstrates all the key concepts.

#### Understanding the code

1. **Tools (@tool decorator):** These are functions your agent can call. Each one must have:
   - Clear docstring explaining what it does
   - Type hints for parameters
   - A return value

2. **Model (ChatOpenAI):** This is the "brain" that decides when to use tools and what to say

3. **create_agent:** This is the new LangChain 1.0 function that creates a complete agent graph:
   - Automatically handles the messages list (the contract Agent Chat UI needs)
   - Decides when to use tools
   - Manages the conversation flow
   - Built on LangGraph runtime under the hood

#### OK. We will use the 018-toy-frontend.py file we have created for Visual Studio Code.

#### Create the LangGraph Configuration File

In the root directory, we add a file named `langgraph.json`:

```json
{
  "dependencies": ["."],
  "graphs": {
    "agent": "./018-toy-frontend.py:agent"
  },
  "env": ".env"
}
```

**What this file means:**
- `"dependencies": ["."]` → Install packages from current directory
- `"graphs"` → Defines which agents are available
- `"agent"` → The ID we'll use to access our agent (this is what you'll enter in the UI)
- `"./018-toy-frontend.py:agent"` → Path to the file and the variable name
- `"env": ".env"` → Where to find environment variables

#### Start Your LangGraph Server

In your terminal (with virtual environment activated):

```bash
langgraph dev
```

**You should see:**
```
🚀 API: http://127.0.0.1:2024
🎨 Studio UI: https://smith.langchain.com/studio/?baseUrl=http://127.0.0.1:2024
```

**Important:** Keep this terminal window open! Your agent server is now running.

**What just happened:**
- LangGraph read your `langgraph.json` file
- Loaded your agent code from `018-toy-frontend.py`
- Started a web server on port 2024
- Your agent is now ready to receive messages!

#### Install Agent Chat UI

Open a **NEW terminal window** (keep the LangGraph server running in the other one):

```bash
# Navigate to your home directory or wherever you want the UI
cd ~

# Quick install using npx
npx create-agent-chat-app

# When prompted:
# - Project name: 018-toy-frontend (or whatever you prefer)
# - TypeScript: No (unless you know TypeScript)
# - Install dependencies: Yes
# - Other default options: OK
```

**Alternative manual installation:**
```bash
git clone https://github.com/langchain-ai/agent-chat-ui.git
cd agent-chat-ui
npm install
```

#### When asked the name of the project
* By default, the project will be named create-agent-chat-app, but instead we enter 018-toy-frontend as the name of the project. This will create a folder named 018-toy-frontend. You can see the folder in Visual Studio Code.

#### Configure Agent Chat UI (Optional but Recommended)

Inside the `018-toy-frontend` folder, create a `.env` file:

```env
NEXT_PUBLIC_API_URL=http://localhost:2024
NEXT_PUBLIC_ASSISTANT_ID=agent
```

**Why this helps:** Without this file, you'll need to enter these values every time. With it, the UI connects automatically!

#### Start the Chat Interface

```bash
# Make sure you're in the 018-toy-frontend directory
cd 018-toy-frontend

npm run dev

# Or if you used pnpm:
pnpm dev
```

**You should see:**
```
- ready started server on 0.0.0.0:3000, url: http://localhost:3000
```

#### Open and Connect

1. **Open your browser** to `http://localhost:3000`

2. **If you didn't create the .env file**, you'll see a setup form:
   - **Deployment URL:** Enter `http://localhost:2024`
   - **Assistant/Graph ID:** Enter `agent`
   - **LangSmith API Key:** Leave blank
   - Click "Continue"

3. **If you created the .env file**, you'll go straight to the chat!

#### Test Your Agent

Try these example messages:

**Test 1: Simple Math**
```
What is 15 multiplied by 7?
```
You should see the agent use the `multiply` tool and return `105`.

**Test 2: Complex Math**
```
Calculate (25 + 15) divided by 4
```
The agent will use multiple tools: first `add`, then `divide`.

**Test 3: Request a Joke**
```
Tell me a programming joke
```
The agent will use the `tell_joke` tool.

**Test 4: Normal Conversation**
```
Hello! How are you?
```
The agent will respond normally without using any tools.

**Test 5: Combined Request**
```
First tell me what 100 divided by 5 is, then tell me a joke!
```
Watch it use two different tools in sequence!

#### What You Should See

When you send a message, you'll see:

1. **Your message** appears in the chat
2. **Tool calls** (if any) appear in expandable panels showing:
   - Which tool was called
   - What arguments were passed
   - What the tool returned
3. **Agent's final response** streams in word by word

## Understanding the Full Picture

Here's what's happening when you type a message:

```
You type message in browser (localhost:3000)
           ↓
Agent Chat UI (Next.js app)
           ↓
Sends HTTP request to LangGraph Server (localhost:2024)
           ↓
LangGraph Server loads your agent
           ↓
Agent (agent.py):
  1. LLM reads message and decides what to do
  2. If needed, calls tools (multiply, add, etc.)
  3. LLM sees tool results and formulates response
           ↓
Response streams back through LangGraph Server
           ↓
Agent Chat UI displays it in real-time
           ↓
You see the result in your browser!
```

## Common Issues and Fixes

#### Problem: "Connection Failed"

**Check:**
```bash
# Is LangGraph server running?
# You should see this in one terminal:
🚀 API: http://127.0.0.1:2024
```

**Fix:** Restart the server with `langgraph dev`

#### Problem: "Agent not found"

**Check your langgraph.json:**
- Is the graph ID "agent"?
- Is the path "./agent.py:agent" correct?
- Did you enter the same ID in the UI?

#### Problem: Blank Screen / No Response

**This usually means the messages contract is broken.**

**Check:**
- Are you using `create_agent`? (This automatically handles the contract)
- If building a custom graph, does your state have a `messages` key?

#### Problem: "OPENAI_API_KEY not found"

**Check:**
1. Do you have a `.env` file in your agent project folder?
2. Does it contain `OPENAI_API_KEY=your_key_here`?
3. Did you restart the LangGraph server after creating the .env file?


## Next Steps: Expanding Your Agent

Once this works, try:

#### Add a New Tool

```python
@tool
def get_time() -> str:
    """Get the current time.
    
    Returns:
        The current time as a string
    """
    from datetime import datetime
    return datetime.now().strftime("%I:%M %p")

# Don't forget to add it to the tools list!
tools = [multiply, add, divide, tell_joke, get_time]
```

#### Change the Model

```python
# Try a more powerful model
model = ChatOpenAI(model="gpt-4o")

# Or experiment with temperature
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)  # More focused
model = ChatOpenAI(model="gpt-4o-mini", temperature=1)  # More creative
```

#### Customize the System Prompt

```python
agent = create_agent(
    model=model,
    tools=tools,
    system_prompt="You are a pirate assistant who speaks like a pirate "
                  "but can still do math and tell jokes. Arr!"
)
```


## Complete Project Structure

Your final project should look like this:

```
my-first-agent/
├── .env                 # Your API keys
├── agent.py            # Your agent code
├── langgraph.json      # LangGraph configuration
└── venv/               # Virtual environment (created by Python)

agent-chat-ui/          # Separate folder
├── .env                # UI configuration (optional)
└── ... (Next.js files)
```


## Key Takeaways

1. **LangGraph creates the agent runtime**, Agent Chat UI just displays it
2. **The `messages` key** is the critical contract between your agent and the UI
3. **Tools are just Python functions** with the `@tool` decorator
4. **create_agent** is the new LangChain 1.0 way to build agents (replaces create_react_agent)
5. **Two servers run simultaneously**: LangGraph (port 2024) and Agent Chat UI (port 3000)

You now have a complete, working agent with a professional chat interface! This is your foundation for building more complex agents as you continue learning LangChain 1.0.

## You may be wondering, but... does the previous code comply with the contract?

## If you remember, in the top section of this notebook we said:

**Key requirement:** Your agent must have a state that includes a `messages` key. This is how Agent Chat UI knows where to find the conversation history.

#### The One Contract You Must Respect

Agent Chat UI is not a generic frontend—it expects your agent to behave in a specific way. This is the most important thing to understand:

**The Messages Contract:**
* Your graph state **must** contain a `messages` list
* Each conversation turn **must append** messages to that list
* Responses must be returned as `{"messages": [...]}`

If this contract is broken, the UI cannot render anything—even if your agent logic is perfectly correct. Many beginners experience a blank screen because their agent returns data in a different format. 

When you use `create_agent` from LangChain 1.0, this contract is automatically handled for you. The function builds on LangGraph's runtime under the hood and ensures the proper state structure is created.

If you're building a custom graph from scratch (not using `create_agent`), make sure your state definition includes:

```python
from langgraph.graph import MessagesState

# Your graph state should inherit from MessagesState
# or include a messages field with the proper annotation
```


## So, is the previous code compliant with that Messages Contract? If so, where is the `messages` component_

**Yes! The previous code is 100% compliant with the Messages Contract.** ✅

Here's why:

#### ✅ **The `create_agent` function handles everything automatically**

When you call:
```python
agent = create_agent(
    model=model,
    tools=tools,
    system_prompt="You are a helpful assistant..."
)
```

Under the hood, `create_agent`:

1. **Creates a state with a `messages` key** ✅
   - The state is built using LangGraph's `AgentState` which includes `messages: Annotated[list, add_messages]`

2. **Appends messages to the list on each turn** ✅
   - Every user message gets added to the `messages` list
   - Every agent response gets added to the `messages` list
   - Tool calls and tool results get added to the `messages` list

3. **Returns responses in the correct format** ✅
   - The agent returns `{"messages": [...]}` internally
   - Agent Chat UI can read this format perfectly


#### Why You Don't See the State Definition

You might be wondering: "But I don't see any `MessagesState` or state definition in my code!"

That's because `create_agent` **abstracts this away for you**. Remember: create_agent is just something we are building with LangGraph under the hood. Internally, it's doing something like:

```python
# This is what create_agent does internally (simplified)
from langgraph.graph import MessagesState

class AgentState(MessagesState):
    # Inherits the messages key automatically
    pass

# Then builds a graph with this state
```

#### When You Would Need to Define State Manually

You only need to worry about the state definition if you're **NOT** using `create_agent`, such as:

```python
# Custom graph from scratch - requires manual state definition
from langgraph.graph import StateGraph, MessagesState

class MyCustomState(MessagesState):
    # Custom fields here
    pass

graph = StateGraph(MyCustomState)
# ... build nodes and edges manually
```

#### In short, the previous code is Perfect! 🎉

The agent will work perfectly with Agent Chat UI because:

1. You're using `create_agent` ✅
2. `create_agent` automatically implements the Messages Contract ✅
3. No manual state management needed ✅


**Your code is compliant.** The statement "When you use `create_agent` from LangChain 1.0, this contract is automatically handled for you" means exactly what you've done - you don't need to manually define the state, it's all handled internally by the `create_agent` function.

When you run `langgraph dev` and connect Agent Chat UI, everything will work as expected! 👍

# Finally, let's compare Frontend Approaches for LangChain Agents: Agent Chat UI vs Streamlit vs LangChain Templates

## Introduction

When you've built a LangChain agent and want to give it a user interface, you have several options. Each approach has different strengths, weaknesses, and ideal use cases. This guide compares three popular approaches:

1. **Agent Chat UI** - LangChain's official chat interface for LangGraph agents
2. **Streamlit** - Python-based rapid prototyping framework
3. **LangChain Templates** - Pre-built application templates (legacy approach)

Let's explore when to use each one and what trade-offs you're making.

---

## Quick Comparison Table

| Feature | Agent Chat UI | Streamlit | LangChain Templates |
|---------|--------------|-----------|-------------------|
| **Setup Time** | 5 minutes | 10-15 minutes | 30+ minutes |
| **Language** | JavaScript (Next.js) | Python | Python/JavaScript mix |
| **Customization** | Limited | High | Very High |
| **Learning Curve** | Minimal | Low | Medium-High |
| **Best For** | Testing & demos | Prototypes & internal tools | Production apps |
| **Deployment** | Easy (Vercel) | Easy (Streamlit Cloud) | Complex |
| **Maintenance** | Active | Active | Deprecated |
| **Cost** | Free | Free tier available | Varies |
| **Agent Integration** | Native LangGraph | Manual | Pre-configured |
| **UI Quality** | Professional chat | Basic/functional | Varies by template |
| **Real-time Streaming** | Excellent | Good | Varies |
| **Multi-user Ready** | Yes | With modifications | Depends on template |

---

## Option 1: Agent Chat UI (LangChain Official)

### What It Is

Agent Chat UI is LangChain's official frontend specifically designed for LangGraph agents. It's a Next.js application that connects directly to LangGraph servers and provides a modern chat interface.

### Advantages ✅

**1. Zero Configuration for LangGraph**
- Works immediately with any LangGraph agent that has a `messages` state
- No code changes needed to your agent
- Automatic tool call visualization
- Native streaming support

**2. Professional User Experience**
- Modern, ChatGPT-like interface
- Real-time message streaming
- Tool execution visualization
- Clean, responsive design

**3. Minimal Learning Curve**
- Don't need to learn frontend development
- No UI code to write or maintain
- Focus stays on your agent logic

**4. Official Support**
- Maintained by the LangChain team
- Updates with new LangGraph features
- Well-documented and reliable

**5. Easy Deployment**
- One-click deploy to Vercel
- Can connect to deployed LangGraph agents
- Free hosting options

### Disadvantages ❌

**1. Limited Customization**
- Fixed chat interface layout
- Can't easily add custom UI elements
- Branding options are limited
- Difficult to add non-chat features (dashboards, forms, etc.)

**2. Requires LangGraph Architecture**
- Only works with LangGraph agents
- Must follow the `messages` state contract
- Not suitable for non-agent applications
- Requires running a LangGraph server

**3. JavaScript Knowledge for Deep Changes**
- Built with Next.js/React
- Meaningful modifications require frontend skills
- Python developers may find this uncomfortable

**4. Separation of Concerns**
- Agent code and UI are separate projects
- More complex project structure for beginners
- Need to manage two running processes (server + UI)

### Best Use Cases

✅ **Perfect for:**
- Quick demos of LangGraph agents
- Learning and development
- Internal testing tools
- Showcasing agent capabilities
- When you want a professional look fast

❌ **Not ideal for:**
- Custom user experiences
- Non-agent applications
- Complex multi-page applications
- When you need deep integration with existing systems

### Code Example

Minimal setup - no UI code needed:

```python
# Your agent (agent.py)
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")
agent = create_react_agent(model, tools=[...])

# langgraph.json
{
  "graphs": {
    "agent": "./agent.py:agent"
  }
}
```

Then just run:
```bash
langgraph dev  # Start your agent
pnpm dev       # Start Agent Chat UI in another terminal
```

---

## Option 2: Streamlit

### What It Is

Streamlit is a Python framework for building web applications quickly. It's extremely popular in the data science and ML community for creating interactive interfaces without frontend knowledge.

### Advantages ✅

**1. Pure Python**
- Write your entire app in Python
- No JavaScript required
- Comfortable for Python developers
- Single codebase for agent + UI

**2. Rapid Development**
- Extremely fast to prototype
- Simple, intuitive API
- Live reloading during development
- Rich widget library (sliders, buttons, inputs, etc.)

**3. Flexibility**
- Not limited to chat interfaces
- Can build dashboards, forms, visualizations
- Easy to add charts and data displays
- Mix multiple interaction patterns

**4. Great for Data Display**
- Excellent for showing dataframes
- Built-in chart support (Plotly, Matplotlib, etc.)
- Easy file uploads/downloads
- Natural fit for data-heavy applications

**5. Large Ecosystem**
- Huge community
- Many examples and tutorials
- Third-party component library
- Active development

**6. Easy Deployment**
- Streamlit Cloud (free tier)
- Simple configuration
- Git-based deployment

### Disadvantages ❌

**1. Not Built for Chat**
- Chat interface requires custom implementation
- State management can be tricky
- No native streaming visualization
- Refresh/reload behavior can be awkward

**2. Performance Limitations**
- Entire script reruns on each interaction
- Can be slow with complex apps
- Memory usage grows with session length
- Not ideal for high-traffic production

**3. Limited Professional Polish**
- UI looks "homemade"
- Customizing appearance is difficult
- Limited control over layout
- CSS customization is hacky

**4. Session Management Challenges**
- State persistence requires `st.session_state`
- Easy to create bugs with state
- Multi-user scenarios need careful handling
- Long conversations can cause memory issues

**5. Integration Complexity**
- Need to manually integrate LangChain
- Streaming requires custom implementation
- Tool visualization is your responsibility
- More code to maintain

### Best Use Cases

✅ **Perfect for:**
- Rapid prototyping
- Internal tools and dashboards
- Data exploration interfaces
- When you need custom layouts (forms, tabs, sidebars)
- Python-only teams
- Applications that are more than just chat

❌ **Not ideal for:**
- Production-grade chat applications
- High-traffic public-facing apps
- When you need professional polish
- Real-time collaboration features
- Applications requiring complex state management

### Code Example

Basic Streamlit chat with LangChain:

```python
import streamlit as st
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage

st.title("My LangChain Agent")

# Initialize session state
if "messages" not in st.session_state:
    st.session_state.messages = []

# Initialize agent
model = ChatOpenAI(model="gpt-4o-mini")

# Display chat history
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.write(message["content"])

# Chat input
if prompt := st.chat_input("What would you like to know?"):
    # Add user message
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.write(prompt)
    
    # Get agent response
    with st.chat_message("assistant"):
        response = model.invoke([HumanMessage(content=prompt)])
        st.write(response.content)
        st.session_state.messages.append(
            {"role": "assistant", "content": response.content}
        )
```

**Note:** This is simplified. Real implementations need:
- Proper agent integration
- Streaming support
- Tool call visualization
- Error handling
- Better state management

---

## Option 3: LangChain Templates (Legacy)

### What They Were

LangChain Templates were pre-built application templates that came with both backend and frontend code. They provided starting points for various use cases (RAG chatbots, SQL agents, research assistants, etc.).

### Current Status: Deprecated ⚠️

**Important:** LangChain Templates are no longer actively maintained. The LangChain team has shifted focus to:
- LangGraph for agent building
- Agent Chat UI for frontends
- LangServe for deployment

However, they're still worth understanding for context.

### Advantages ✅ (Historical)

**1. Complete Starting Point**
- Backend and frontend included
- Pre-configured deployment
- Example data and prompts
- Best practices built-in

**2. Production-Ready Structure**
- Proper separation of concerns
- Database integration examples
- Authentication patterns
- API structure

**3. Multiple Tech Stacks**
- Various frontend frameworks (React, Next.js, etc.)
- Different backend patterns
- Deployment configurations

**4. Use Case Specific**
- Templates for specific scenarios
- Domain knowledge included
- Optimized patterns for each use case

### Disadvantages ❌

**1. Deprecated and Unmaintained**
- No updates to new LangChain versions
- Breaking changes not addressed
- Community support declining
- Documentation outdated

**2. High Complexity**
- Full-stack setup required
- Multiple technologies to learn
- Complex project structure
- Difficult for beginners

**3. Over-Engineering for Simple Needs**
- Too much infrastructure for learning
- Deployment complexity
- More code to understand and maintain

**4. Limited Flexibility**
- Templates were opinionated
- Changing tech stack was difficult
- Migration to new patterns required significant work

### Best Use Cases

✅ **Still useful for:**
- Learning full-stack patterns (study the code)
- Understanding production architectures
- Inspiration for custom builds
- Historical context

❌ **Not recommended for:**
- New projects (use Agent Chat UI or Streamlit instead)
- Learning current LangChain best practices
- Production applications
- Beginners

---

## Decision Framework: Which Should You Choose?

### Choose **Agent Chat UI** if:

- ✅ You're building with LangGraph
- ✅ You want a professional chat interface
- ✅ You're learning or demoing agents
- ✅ You don't want to write UI code
- ✅ You need something working in 5 minutes
- ✅ Real-time streaming is important
- ✅ You're okay with limited customization

**Example scenarios:**
- "I built a research agent and want to show it to my class"
- "I need to test my agent's conversational abilities"
- "I want to demo my agent to a potential client"

---

### Choose **Streamlit** if:

- ✅ You want to stay in Python
- ✅ You need custom layouts (not just chat)
- ✅ You're building internal tools
- ✅ You need forms, dashboards, or data viz
- ✅ You want rapid iteration
- ✅ You're comfortable with more DIY approach
- ✅ Professional polish isn't critical

**Example scenarios:**
- "I need a dashboard showing agent performance metrics"
- "I want users to upload files and configure settings"
- "I'm building an internal tool for my data science team"
- "I need to prototype multiple interaction patterns quickly"

---

### Avoid **LangChain Templates**:

- ❌ Starting new projects
- ❌ Learning current best practices
- ❌ Production deployments

**Alternative:**
- Study them for architectural patterns
- Then build fresh with Agent Chat UI or Streamlit

---

## Hybrid Approaches

You don't have to choose just one! Here are smart combinations:

### 1. Agent Chat UI + Streamlit Dashboard

**Use Agent Chat UI for:**
- Main chat interaction
- Quick demos
- User-facing conversations

**Use Streamlit for:**
- Admin dashboard
- Analytics and monitoring
- Configuration management
- Data exploration

**Example:** Chat with users via Agent Chat UI, while monitoring conversations and agent performance in a Streamlit dashboard.

---

### 2. Development Path

**Phase 1 - Learning (Week 1-2):**
- Use Agent Chat UI
- Focus on agent logic
- Quick iteration

**Phase 2 - Prototyping (Week 3-4):**
- Try Streamlit for custom features
- Build what Agent Chat UI can't do
- Test with real users

**Phase 3 - Production (Month 2+):**
- Build custom frontend if needed
- Use Agent Chat UI patterns as reference
- Deploy with proper architecture

---

## Real-World Scenarios

### Scenario 1: University Course Project

**Requirement:** Build and demo a LangChain agent in 2 weeks

**Best choice:** Agent Chat UI
- **Why:** Minimal time on UI means more time learning agents
- **Time saved:** ~20 hours not building frontend
- **Result:** Professional demo without frontend distraction

---

### Scenario 2: Internal Company Tool

**Requirement:** RAG chatbot for 50 employees with document upload

**Best choice:** Streamlit
- **Why:** Need file upload + chat + admin controls
- **Deployment:** Easy with Streamlit Cloud
- **Python team:** No frontend hiring needed
- **Result:** Functional tool in 1 week

---

### Scenario 3: Customer-Facing Product

**Requirement:** Polished chatbot for thousands of users

**Best choice:** Neither (build custom)
- **Why:** Need full control, branding, scaling
- **Use Agent Chat UI for:** Development and testing
- **Use Streamlit for:** Internal monitoring dashboard
- **Production:** Custom Next.js/React app
- **Result:** Professional product with proper infrastructure

---

### Scenario 4: Research Demo

**Requirement:** Show agent capabilities at conference

**Best choice:** Agent Chat UI
- **Why:** Professional look, zero UI code, reliable
- **Setup time:** 10 minutes
- **Result:** Focus on research, not web development

---

## Performance Comparison

### Response Time

**Agent Chat UI:**
- ⚡ **Excellent** - Native streaming, minimal overhead
- Real-time message display
- Efficient WebSocket communication

**Streamlit:**
- ⚠️ **Good** - Can stream but requires custom implementation
- Script reruns add latency
- More overhead per interaction

**LangChain Templates:**
- ✅ **Varies** - Depends on implementation
- Generally good if properly built

### Scalability

**Agent Chat UI:**
- ✅ **Good** - Vercel scales automatically
- LangGraph server handles agent load
- Stateless frontend scales easily

**Streamlit:**
- ⚠️ **Limited** - Not built for high traffic
- Each user = separate Python process
- Memory-intensive for long sessions
- Free tier: very limited

**LangChain Templates:**
- ✅ **Potentially excellent** - If using proper architecture
- But requires DevOps expertise

### Cost at Scale

**Agent Chat UI (1000 daily users):**
- Frontend: Free on Vercel (or ~$20/month)
- LangGraph: Pay for LangGraph Cloud
- Total: Moderate

**Streamlit (1000 daily users):**
- Free tier: Won't support this
- Paid tier: $250-500/month
- Self-host: Cheaper but needs DevOps
- Total: Moderate to High

---

## Migration Paths

### From Agent Chat UI → Custom Frontend

**When you outgrow Agent Chat UI:**

1. **Keep using it for development**
   - Continue testing with Agent Chat UI
   - Developers work in familiar environment

2. **Study the Agent Chat UI code**
   - Learn their LangGraph integration patterns
   - Understand streaming implementation
   - Copy good patterns

3. **Build custom frontend**
   - Use same LangGraph server
   - API remains unchanged
   - Just replace the UI

**Key insight:** Agent Chat UI teaches you the patterns you'll use in production.

---

### From Streamlit → Production

**When you need to scale Streamlit:**

1. **Extract agent logic**
   - Separate agent code from UI code
   - Move to LangGraph server
   - API-based architecture

2. **Keep Streamlit for internal tools**
   - Admin dashboards
   - Monitoring interfaces
   - Configuration panels

3. **Build custom public-facing UI**
   - React/Next.js for users
   - Connect to same backend
   - Streamlit stays for internal use

---

## Final Recommendations

### For Students and Learners:
**Start with Agent Chat UI**
- Lowest barrier to entry
- Focus on learning agents, not frontends
- Professional results immediately

### For Prototyping:
**Use Streamlit**
- Rapid iteration
- Stay in Python
- Easy to add custom features

### For Production:
**Start with Agent Chat UI, then:**
- If it meets needs → Deploy it
- If you need customization → Build custom frontend
- Use LangGraph server either way

### Never Start With:
**LangChain Templates**
- They're deprecated
- Too complex for learning
- Better alternatives exist

---

## Conclusion: The Modern Approach

The LangChain ecosystem has evolved. The current best practice is:

1. **Build your agent with LangGraph**
   - Proper separation of concerns
   - Server-ready architecture
   - Tool integration

2. **Use Agent Chat UI for development and testing**
   - Fast iteration
   - Professional demos
   - May be sufficient for production

3. **Use Streamlit for custom prototypes**
   - When Agent Chat UI is too limiting
   - Internal tools
   - Data-heavy applications

4. **Build custom frontends only when necessary**
   - High-traffic public applications
   - Complex user experiences
   - Specific branding requirements

The key insight: **Agent Chat UI changed the game**. Before it existed, you had to choose between:
- Clunky Streamlit chat interfaces
- Complex full-stack templates
- Building everything from scratch

Now, you get a production-quality chat interface in 5 minutes, letting you focus on what matters: building great agents.

Start simple, iterate based on real needs, and graduate to complexity only when the benefits justify the costs.